In [1]:
import plotly.graph_objects as go
import plotly.express as px

import numpy as np
import pandas as pd

In [32]:
quebec_path = "../data/quebec/Quebec.csv"
headers = ["Region",  "Total Number of Cases", "Date of Infection", "Lat", "Long"]
df = pd.read_csv(quebec_path)
regions = df[:19]
dates = list(regions)[1:]

lat_long = [(48.333333, -68.666667), (49.8667, -71.75), (47.333333, -71.5), (47, -73), (45.481556, -71.667917),
               (45.5, -73.566667), (45.783333, -75.083333), (48.233333, -79.016667), (50.866667, -65.816667),
               (56.166667, -74.416667), (48.683333, -65.4), (46.483333, -70.616667), (45.583333, -73.75),
               (46.766667, -73.833333), (46.433333, -74.983333), (45.383333, -73.1), (46.283333, -72.066667)]

In [33]:
stacked_df = pd.DataFrame(columns=headers)
for ix,row in regions.iterrows():
    region = [list(row)[0]]*(len(row)-1)
    if ix < 17:
        lat = [lat_long[ix][0]]*(len(row)-1)
        long = [lat_long[ix][1]]*(len(row)-1)
    else : 
        lat = ["na"]*(len(row)-1)
        long = ["na"]*(len(row)-1)
    num_cases = list(row)[1:]
    region_list = [region, num_cases, dates, lat, long]
    temp = pd.DataFrame(list(map(list, zip(*region_list))), columns = headers)
    stacked_df =stacked_df.append(temp,  ignore_index=True)


In [34]:
stacked_df

,Region,Total Number of Cases,Date of Infection,Lat,Long
0,01-Bas-Saint-Laurent,0.0,13th,48.3333,-68.6667
1,01-Bas-Saint-Laurent,0.0,14th,48.3333,-68.6667
2,01-Bas-Saint-Laurent,0.0,15th,48.3333,-68.6667
3,01-Bas-Saint-Laurent,0.0,16th,48.3333,-68.6667
4,01-Bas-Saint-Laurent,0.0,17th,48.3333,-68.6667
...,...,...,...,...,...
185,Hors Quebec,2.0,18th,na,na
186,Hors Quebec,2.0,19th,na,na
187,Hors Quebec,2.0,20th,na,na
188,Hors Quebec,2.0,21st,na,na


In [16]:
total_cases = list(df.iloc[19])[1:]

In [17]:
fig = px.bar(stacked_df, x="Date of Infection", y="Total Number of Cases", color = "Region", color_discrete_sequence= px.colors.qualitative.Light24)
fig.add_trace(go.Scatter(x=dates, y=total_cases,
                    mode='lines',
                    name='Total'))
fig.show()
with open('../plotly/Total_cases.html', 'w') as f:
    f.write(fig.to_html(include_plotlyjs='cdn'))

In [18]:
new_cases = list(df.iloc[20])[2:]

In [19]:
fig = go.Figure(data=go.Scatter(x=dates[1:], y=new_cases,
                    mode='lines',
                    name='New Cases'))
fig.update_layout(title='New Cases Each Day',
                   xaxis_title='Day',
                   yaxis_title='Number of New Cases')
fig.show()
with open('../plotly/new_cases.json', 'w') as f:
    f.write(fig.to_json())

In [20]:
headers = ["Results",  "People Tested", "Lab Report Date"]
positive = list(df.iloc[20])[1:]
negative = list(df.iloc[23])[1:]

In [22]:
fig = go.Figure(data=[
    go.Bar(name='Negative', x=dates[2:], y=negative[2:]),
    go.Bar(name='Positive', x=dates[2:], y=positive[2:])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_layout(
                   xaxis_title='Lab Report Day',
                   yaxis_title='Peope Tested')
fig.show()
with open('../plotly/tested.json', 'w') as f:
    f.write(fig.to_json())

In [35]:
lastday_df = stacked_df.loc[stacked_df['Date of Infection'] == dates[-1]][:17]
del lastday_df['Date of Infection']

In [38]:
token = "pk.eyJ1Ijoibmlja2JlbnQiLCJhIjoiY2s4MjFyejMwMGRhMDNqcnU1aG0wZTdrZSJ9.t7FEmorhX4kygVCGQw17Bg"
px.set_mapbox_access_token(token)
fig = px.scatter_mapbox(lastday_df, lat="Lat", lon="Long",     color="Region", size="Total Number of Cases",
                        mapbox_style="carto-positron", color_continuous_scale=px.colors.cyclical.IceFire, size_max=30, zoom=4)
fig.show()
with open('../plotly/map.json', 'w') as f:
    f.write(fig.to_json())

In [42]:
fig = px.scatter_geo(stacked_df, lat="Lat", lon="Long",     color="Region", size="Total Number of Cases",
                     animation_frame="Date of Infection", projection="natural earth",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=30)
fig.show()
with open('../plotly/map_timeline.json', 'w') as f:
    f.write(fig.to_json())

In [79]:
fig = px.area(stacked_df, x="Date", y="Number of Cases", color = "Region", color_discrete_sequence= px.colors.qualitative.Light24)
fig.show()